## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Mataura,NZ,2022-05-02 00:18:14,-46.1927,168.8643,62.47,72,13,4.18,few clouds
1,1,Cherskiy,RU,2022-05-02 00:18:15,68.7500,161.3000,28.45,85,16,11.72,few clouds
2,2,Ancud,CL,2022-05-02 00:18:15,-41.8697,-73.8203,50.32,91,41,4.14,scattered clouds
3,3,Wanning,CN,2022-05-02 00:18:16,18.8003,110.3967,69.58,94,100,16.08,overcast clouds
4,4,Jasper,US,2022-05-02 00:18:16,37.2001,-94.3502,71.11,42,40,5.75,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 74
What is the maximum temperature you would like for your trip? 76


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
7,7,Oriximina,BR,2022-05-02 00:18:18,-1.7656,-55.8661,75.65,94,63,2.57,broken clouds
9,9,Saint-Philippe,RE,2022-05-02 00:10:19,-21.3585,55.7679,74.79,84,2,2.91,clear sky
52,52,Marawi,PH,2022-05-02 00:19:35,7.9986,124.2928,74.73,87,99,0.51,overcast clouds
79,79,Bundaberg,AU,2022-05-02 00:17:47,-24.8500,152.3500,75.49,78,75,13.80,light rain
133,133,Hilton Head Island,US,2022-05-02 00:18:38,32.2163,-80.7526,74.98,78,0,10.36,clear sky
143,143,Pacifica,US,2022-05-02 00:22:16,37.6138,-122.4869,75.43,69,20,27.63,haze
201,201,Yeppoon,AU,2022-05-02 00:23:44,-23.1333,150.7333,74.98,79,97,15.52,overcast clouds
206,206,Itupiranga,BR,2022-05-02 00:23:46,-5.1347,-49.3267,74.61,92,26,1.59,scattered clouds
214,214,Belize,BZ,2022-05-02 00:23:50,17.2500,-88.7500,75.16,97,100,3.38,overcast clouds
229,229,Saint George,US,2022-05-02 00:23:57,37.1041,-113.5841,74.62,15,0,27.63,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))

35


In [6]:
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities2_df=preferred_cities_df.dropna(axis="index", how="any")
print(len(preferred_cities2_df))

35


In [8]:
preferred_cities2_df.isnull().sum()

City_ID        0
City           0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities2_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
7,Oriximina,BR,75.65,broken clouds,-1.7656,-55.8661,
9,Saint-Philippe,RE,74.79,clear sky,-21.3585,55.7679,
52,Marawi,PH,74.73,overcast clouds,7.9986,124.2928,
79,Bundaberg,AU,75.49,light rain,-24.8500,152.3500,
133,Hilton Head Island,US,74.98,clear sky,32.2163,-80.7526,
143,Pacifica,US,75.43,haze,37.6138,-122.4869,
201,Yeppoon,AU,74.98,overcast clouds,-23.1333,150.7333,
206,Itupiranga,BR,74.61,scattered clouds,-5.1347,-49.3267,
214,Belize,BZ,75.16,overcast clouds,17.2500,-88.7500,
229,Saint George,US,74.62,clear sky,37.1041,-113.5841,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
print(len(hotel_df))
hotel_df.dtypes
hotel_df.isna().sum()

35


City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [27]:
# 7. Drop the rows where there is no Hotel Name.
hotel2_df=hotel_df.dropna(axis="index", how="any")
print(len(hotel2_df))

35


In [22]:
hotel2_df


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
7,Oriximina,BR,75.65,broken clouds,-1.7656,-55.8661,Pousada Tia Amélia
9,Saint-Philippe,RE,74.79,clear sky,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
52,Marawi,PH,74.73,overcast clouds,7.9986,124.2928,Derogongan Residence
79,Bundaberg,AU,75.49,light rain,-24.8500,152.3500,Takalvan Motel
133,Hilton Head Island,US,74.98,clear sky,32.2163,-80.7526,Disney’s Hilton Head Island Resort
143,Pacifica,US,75.43,haze,37.6138,-122.4869,"Pacifica Lighthouse, Trademark Collection by W..."
201,Yeppoon,AU,74.98,overcast clouds,-23.1333,150.7333,Villa Mar Colina
206,Itupiranga,BR,74.61,scattered clouds,-5.1347,-49.3267,Hotel Panorama
214,Belize,BZ,75.16,overcast clouds,17.2500,-88.7500,Villa San Juan
229,Saint George,US,74.62,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel2_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel2_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel2_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))